# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,Unnamed: 0,Latitude,Longitude,City,Temp,Humidity,Wind Speed,Cloudiness,Date
0,0,71.29,-156.79,barrow,24.80,68.0,13.87,90.0,1.604196e+09
1,1,44.36,77.98,saryozek,36.52,66.0,2.35,0.0,1.604196e+09
2,2,22.89,-109.91,cabo san lucas,80.60,74.0,5.82,20.0,1.604196e+09
3,3,42.60,-73.97,albany,35.01,84.0,5.01,0.0,1.604195e+09
4,4,69.30,139.90,deputatskiy,-9.62,90.0,5.37,85.0,1.604196e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)

In [24]:
cities_df = cities_df.dropna()
len(cities_df)

550

In [25]:
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
vaca_df = cities_df.loc[(cities_df["Temp"]<82) & (cities_df["Temp"]>74) & (cities_df["Wind Speed"]<10)&
                        (cities_df["Cloudiness"]==0)]
vaca_df
vaca_df.dropna(how="any")
vaca_df.count()

Unnamed: 0    6
Latitude      6
Longitude     6
City          6
Temp          6
Humidity      6
Wind Speed    6
Cloudiness    6
Date          6
dtype: int64

In [29]:
vaca_df = hotel_df
hotel_df

,Unnamed: 0,Latitude,Longitude,City,Temp,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
37,43,13.17,44.58,lahij,77.47,72.0,7.61,0.0,1.604077e+09,
253,513,21.64,69.61,porbandar,81.70,52.0,1.97,0.0,1.604077e+09,
310,653,22.57,59.53,sur,74.73,66.0,5.06,0.0,1.604077e+09,
361,771,24.69,46.72,riyadh,75.20,29.0,3.36,0.0,1.604077e+09,
397,865,17.02,54.09,salalah,80.60,61.0,3.36,0.0,1.604077e+09,
559,1397,-16.23,39.91,angoche,79.84,74.0,9.98,0.0,1.604077e+09,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,Latitude,Longitude,City,Temp,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
37,43,13.17,44.58,lahij,77.47,72.0,7.61,0.0,1.604077e+09,
253,513,21.64,69.61,porbandar,81.70,52.0,1.97,0.0,1.604077e+09,
310,653,22.57,59.53,sur,74.73,66.0,5.06,0.0,1.604077e+09,
361,771,24.69,46.72,riyadh,75.20,29.0,3.36,0.0,1.604077e+09,
397,865,17.02,54.09,salalah,80.60,61.0,3.36,0.0,1.604077e+09,


In [30]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key}

for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    name_address = requests.get(url, params=params)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "AcnaJv2rEo3bA6vwfVZSjpD35_yc4znez17-Z7UxELkJqbR-lz-jNA0gAd-vkqxOnND3Hq5-Ydxilyccb1LVL7oqHQXeGoAgAnDwO1o1XvNV5MvDCjjkqH8ChZi6uo9CTNE_VPvkuCoKoed9AjdU5QNH4dz4Pth3LOjc6TfZCUXPGdusYAvdWkiaVBRZCoRy9vSQGJFVd3BA34RQ5ATv8PnbPQaRU_yqvX0riB5muCKt_h9tXp8CHn3ZltMcaQflieCAbZK7Z1i1mUFLoD6uCn3GWVW52vnHii7Kf1Zt4jJwxO5J9uGrYpCZ1-75LWjL_1FPl07R1EQdmk1cwlYpy4lN5feWnBqdJqcNIDn-2vtpUo5jY6KXsBjBbWNR85FPUpVh15Ysjfb8Lo42oZGLN3k1WZoSNnW6ZJubaByqumwOP9Zso3F8Di5PBzBL-Jvl",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 21.6314149,
                    "lng": 69.6090348
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.63281432989272,
                        "lng": 69.61047

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.5664354,
                    "lng": 59.4958585
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.56785167989272,
                        "lng": 59.49709562989273
                    },
                    "southwest": {
                        "lat": 22.56515202010728,
                        "lng": 59.49439597010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Sur Plaza Hotel",
            "photos": [
                {
                    "height": 4032,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/102820958213968235867\">d.flachsmann</

{
    "html_attributions": [],
    "next_page_token": "AcnaJv2r2DZNDzzhhpXA-ZEhMFdEAS1Bf5V2WnuatznlhKxG3FI1hDk5y3HOqgR5R0Xod6Wwg8aOi1HzC6kJHvA8pVEWglCUp8KpEwR9qaRYxThhEmpCB1kq3c2Q1CP48LKx1PNvgtmasuY5tGoJkLjdhKISRonyKgz7nDFJyFuX8oTBoQ5kWV_t2WkE8Rp7jJVIod8XiirM9mfk2nik3rE6aMGdvl7eUjDUXroWOjYTlHnZIuVY7yyr0D1cFIR0IfEW39r4DlA8aq82XCSSphjWO1LZ45ud7Tnqyx2iECayWRs9ZrN5UiglXpuf-0KZ0tL-7AyuqS7C_pWXNxGfAQ1T-wXCtGey9rT-NgTVZFT_1WtDoxvc6KUUcGTwqJM-I3_JOugBiIK9KKbVMH5uJ8CJvM8yKmdwLSdRVrqdG5BS3tFjBxXc5ryabUtll6cn",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.7114911,
                    "lng": 46.6745231
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.71218262989272,
                        "lng": 46.67617147989272
                    },
                    "southwest": {
                        "lat": 24.7094829701072

{
    "html_attributions": [],
    "next_page_token": "AcnaJv3BKAqNypeDxzC155Ry7ejcH-yobzsFSjffauqyge9u_-ImgsKUuXdjOJABfItSQUFTobO-b-MGGUMl4PaWXWq3194x3CbB7LygYBB_VWiNVptiAlbs_-KncBodf0JW_V6KxeCP3ZSDwulwRQQUoRhyEFGTvAyL58WJC-hG7cHq3911xGrT_8sYRzeGmTi9ViyQSV6ZVFi-0wN3iyBvhZkWLf6RbVP_1mRNlCBrbfV-5bW50YPa0N2LdZBNvqBRsmWLwMQ0orYb7nzZ9xi3PSxiaIlcQZEQFS81sf5uxmcMKGoB3oEQahsZ60paDSO8NHsaQaIZuxUYzY34MwxHGvjJ44zj1maUHWAe_3hgXoKpvyKN4xRuXsWyBFRjtqSS6LbMjJsAUAFexur9oXSB_N4aENs6VC6lMxrWOEOIdOw06DI3-okHLLryqEs-",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 17.022848,
                    "lng": 54.064693
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.02417637989272,
                        "lng": 54.06600327989272
                    },
                    "southwest": {
                        "lat": 17.02147672010728,

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -16.2259238,
                    "lng": 39.9058279
                },
                "viewport": {
                    "northeast": {
                        "lat": -16.22468642010728,
                        "lng": 39.90708597989272
                    },
                    "southwest": {
                        "lat": -16.22738607989272,
                        "lng": 39.90438632010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Guest House HESADA",
            "photos": [
                {
                    "height": 2322,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/108692098884708211150\">Hetul Sa

In [31]:
hotel_df

,Unnamed: 0,Latitude,Longitude,City,Temp,Humidity,Wind Speed,Cloudiness,Date,Hotel Name
37,43,13.17,44.58,lahij,77.47,72.0,7.61,0.0,1.604077e+09,
253,513,21.64,69.61,porbandar,81.70,52.0,1.97,0.0,1.604077e+09,Lords Inn Porbandar
310,653,22.57,59.53,sur,74.73,66.0,5.06,0.0,1.604077e+09,Sur Plaza Hotel
361,771,24.69,46.72,riyadh,75.20,29.0,3.36,0.0,1.604077e+09,Four Seasons Hotel Riyadh At Kingdom Center
397,865,17.02,54.09,salalah,80.60,61.0,3.36,0.0,1.604077e+09,Salalah Gardens Hotel
559,1397,-16.23,39.91,angoche,79.84,74.0,9.98,0.0,1.604077e+09,Guest House HESADA


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

#<dt>Country</dt><dd>{Country}</dd> (I didn't use country - missed that part)

In [33]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_df.iterrows()]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))